In [ ]:
import os
import json

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from dotenv import load_dotenv
load_dotenv()

In [ ]:
api_key = os.getenv("API_KEY")
client_secrets_file = os.getenv("CLIENT_SECRET")

api_service_name = "youtube"
api_version = "v3"
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

In [ ]:
def get_credentials():
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)
    credentials = flow.run_local_server(port=8000, prompt="consent")
    return credentials

def get_playlists(youtube_service):
    response = youtube_service.playlists().list(
        part="snippet,contentDetails",
        mine=True
    ).execute()
    return response["items"]

def get_videos(youtube_service, playlist_id):
    response = youtube_service.playlistItems().list(
        part="snippet,contentDetails",
        maxResults=50,
        playlistId=playlist_id
    ).execute()

    items = response["items"]
    video_ids = [item["contentDetails"]["videoId"] for item in items]

    videos_response = youtube_service.videos().list(
        part="snippet,contentDetails",
        id=",".join(video_ids)
    ).execute()

    videos = []
    for video in videos_response["items"]:
        video_data = {
            "title": video["snippet"]["title"],
            "duration": video["contentDetails"]["duration"]
        }
        videos.append(video_data)

    return videos

def main():
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    # Get credentials and create an API client
    credentials=get_credentials()

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    playlists = get_playlists(youtube)
    for playlist in playlists:
        print(playlist["snippet"]["title"])
        videos = get_videos(youtube, playlist["id"])
        print("Videos:")

        for video in videos:
            print(video["snippet"]["title"])
        


if __name__ == "__main__":
    main()
